In [1]:
import ee
import geemap

In [2]:
ee.Authenticate()
ee.Initialize(project = 'ee-yuvrajaste1107')

In [3]:
#ROI
roi = ee.Geometry.Point([72.8777, 19.0760]) # Mumbai Co-ordinates 72.8777, 19.0760

location = roi.centroid().coordinates().getInfo()[::-1]

def apply_scale_factors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBand = image.select('ST_B10').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True) \
    .addBands(thermalBand, None, True)

image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2001-01-01', '2017-12-31') \
    .filterBounds(roi) \
    .map(apply_scale_factors) \
    .sort('CLOUD_COVER') \
    .first()

ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename(['ndvi'])

mndwi = image.normalizedDifference(['SR_B3', 'SR_B6']).rename(['mndwi'])

images = image.addBands(ndvi).addBands(mndwi)

ndwi = image.normalizedDifference(['SR_B3', 'SR_B6']).rename(['ndwi'])

visualization_nir_rgb = {
    'bands': ['SR_B5', 'SR_B4', 'SR_B3'],
    'min': 0.0,
    'max': 0.3,
}


Map = geemap.Map()
Map.centerObject(image, 8)


Map.addLayer(image, visualization_nir_rgb, 'NIR, Red, Green')
Map


Map(center=[18.78738423523505, 72.37901176061042], controls=(WidgetControl(options=['position', 'transparent_b…

In [4]:
lc = ee.ImageCollection('projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS') \
    .filter(ee.Filter.date('2017-01-01', '2017-12-31')) \
    .mosaic() \
    .remap([1,2,4,5,7,8,9,10,11],[1,2,3,4,5,6,7,8,9]) \
    .rename('lc')
print(lc.getInfo())


{'type': 'Image', 'bands': [{'id': 'lc', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 9}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [5]:
label = 'lc'
bands = ['SR_B1','SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7',  'ndvi', 'mndwi']

sample = images.addBands(lc).sample(**{
    'region': image.geometry(),
    'numPixels' : 5000,
    'scale': 30,
    'seed' : 1
})

sample = sample.randomColumn()
training = sample.filter('random <= 0.8')
validation = sample.filter('random > 0.8')

trainedClassifier = ee.Classifier.smileRandomForest(10).train(**{
    'features': training,
    'classProperty': label,
    'inputProperties': bands
})

model = images.classify(trainedClassifier)

In [6]:
dict = {
    "names":[
        "Water",
        "Trees",
        "Flooded Vegetation",
        "Crops",
        "Built Area",
        "Bare Ground",
        "snow/ice",
        "clouds",
        "Rangeland"

    ],
    "colors": [
        "#1A5BAB",
        "#358221",
        "#87D19E",
        "#FFD85C",
        "#ED022A",
        "#EDE9E4",
        "#F2FAFF",
        "#C8C8C8",
        "#C6AD8D"
    ]
}

In [7]:
Map.addLayer(lc.clip(image.geometry()), {'min':1, 'max':9, 'palette': dict['colors']}, 'ESRI LULC 10m 2017')
Map.addLayer(model, {'min':1, 'max':9, 'palette': dict['colors']}, 'Classified 2017')
Map.centerObject(image, 8)
Map


Map(bottom=29585.0, center=[18.78738423523504, 72.37901176061048], controls=(WidgetControl(options=['position'…